# Theano Print

In [ ]:
import numpy as np
import theano
import theano.tensor as T
rng = np.random
# Training data
N = 400
feats = 784
D = (rng.randn(N, feats).astype(theano.config.floatX), rng.randint(size=N,low=0, high=2).astype(theano.config.floatX))
training_steps = 10000
# Declare Theano symbolic variables
x = T.matrix("x")
y = T.vector("y")
w = theano.shared(rng.randn(feats).astype(theano.config.floatX), name="w")
b = theano.shared(np.asarray(0., dtype=theano.config.floatX), name="b")
x.tag.test_value = D[0]
y.tag.test_value = D[1]
# Construct Theano expression graph
p_1 = 1 / (1 + T.exp(-T.dot(x, w)-b)) # Probability of having a one
prediction = p_1 > 0.5 # The prediction that is done: 0 or 1
# Compute gradients
xent = -y*T.log(p_1) - (1-y)*T.log(1-p_1) # Cross-entropy
cost = xent.mean() + 0.01*(w**2).sum() # The cost to optimize
gw,gb = T.grad(cost, [w,b])
# Training and prediction function
train = theano.function(inputs=[x,y], outputs=[prediction, xent], updates=[[w, w-0.01*gw], [b, b-0.01*gb]], name = "train")
predict = theano.function(inputs=[x], outputs=prediction, name = "predict")

In [ ]:
theano.printing.pprint(prediction) 

In [ ]:
theano.printing.pydotprint(prediction, outfile="logreg_pydotprint_prediction.png", var_with_name_simple=True)  

# CoreNLP

In [ ]:
sent_data = []
cnt = 0
import time
start = time.time()
from pycorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('http://localhost:9000')
# total 302736
with open("./data/train1.ner.txt", "r") as f:
    for sent in f:
        cnt += 1
        if(cnt % 1e2==0):
            print (cnt, time.time()-start)
        sent = sent.replace("\n", "")
        res = nlp.annotate(sent, properties={'annotators':"pos", 'outputFormat':'json'})
        res = res["sentences"]
        #print(sent)
        noun_list = []
        for word in res[0]['tokens']:
            if(word['pos'][0]=='N'):
                noun_list.append(word['word'])
        sent_format = (sent, noun_list)
        sent_data.append(sent_format)
import json
with open("./data/sents.json", "w") as f:
    json.dump(sent_data, f)

100 1.4687469005584717
